This is version 2 of my python ML script for the titanic submission. 

Your Best Entry!Your most recent submission scored 0.73923, which is an improvement of your previous score of 0.71291. Great job!

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Updated file paths
train_path = r"Path.csv"
test_path = r"Path.csv"

# Load the datasets into Pandas DataFrames
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

# Handle missing values in Age and Fare
train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
train_data['Fare'].fillna(train_data['Fare'].median(), inplace=True)

test_data['Age'].fillna(test_data['Age'].median(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].median(), inplace=True)

# Encode categorical features
train_data['Sex'] = train_data['Sex'].map({'male': 0, 'female': 1})
test_data['Sex'] = test_data['Sex'].map({'male': 0, 'female': 1})

train_data = pd.get_dummies(train_data, columns=['Embarked'], drop_first=True)
test_data = pd.get_dummies(test_data, columns=['Embarked'], drop_first=True)

# Feature Scaling
scaler = StandardScaler()
numerical_features = ['Age', 'Fare']
train_data[numerical_features] = scaler.fit_transform(train_data[numerical_features])
test_data[numerical_features] = scaler.transform(test_data[numerical_features])

# Feature Engineering: Combine 'Pclass' and 'Age' to create a new feature 'Class_Age'
train_data['Class_Age'] = train_data['Pclass'] * train_data['Age']
test_data['Class_Age'] = test_data['Pclass'] * test_data['Age']

# Separate features and target variable
X = train_data.drop(['Survived', 'PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
y = train_data['Survived']

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training and evaluation using cross-validation
models = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SVC(probability=True)
]

for model in models:
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    print(f"\nCross-validated Accuracy for {model.__class__.__name__}: {np.mean(scores)}")

# Train the best-performing model on the full training set
best_model = RandomForestClassifier()
best_model.fit(X, y)

# Make predictions on the test set
X_test = test_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
test_predictions = best_model.predict(X_test)

# Create a submission DataFrame
submission_df = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': test_predictions})

# Save the submission to a CSV file
submission_df.to_csv('titanic_submission.csv', index=False)
